## Defining a Model

A Pydantic model is a Python class that defines the structure of your data using type hints. It automatically checks and converts input values to the correct types.

- from pydantic import BaseModel import the Pydantic base class.

- class UserProfile(BaseModel) define a typed data model.

- name: str, age: int, email: str fields with type annotations.

- user = UserProfile(...) construct a model hence triggers validation/coercion.

In [ ]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    name: str
    age: int
    email: str

user = UserProfile(name="Liam Svensson", age=40, email="liam.svensson@example.com")
print(user)

## Handling Default Values

Pydantic lets you set default values for fields and mark others as required. If a field has a default value, it’s automatically used when that field is missing in input data.

- age: int = 43 default value used when age is omitted.

- is_active: bool = True default boolean flag.

- user = UserProfile(...) missing fields with defaults are filled automatically.

In [ ]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    name: str
    age: int = 43
    email: str
    is_active: bool = True

user = UserProfile(name="Sofia Moretti", email="sofia.moretti@example.com")
print(user)

## Field Validators

Field validators let you define custom validation rules for individual fields. They are useful when you need to enforce constraints beyond type checking.

- @field_validator('age'): Decorator to add a custom validation function for the age field.

- def check_age(cls, value): Function that checks if age is at least 18.

- raise ValueError(...): Raises an error if the age condition fails.

- UserProfile(...): Creating an instance triggers the validation.

In [ ]:
from pydantic import BaseModel, field_validator

class UserProfile(BaseModel):
    name: str
    age: int
    email: str

    @field_validator('age')
    def check_age(cls, value):
        if value < 18:
            raise ValueError('Age must be at least 18')
        return value

UserProfile(name="Noah Müller", age=17, email="noah.muller@example.com")

## Model Validators

Model validators let you define validation rules that involve multiple fields or the model as a whole.

- @model_validator(mode='after'): Runs validation after the model is fully initialized.

- def passwords_match(cls, model): Checks if password and confirm_password match.

- raise ValueError(...): Raises an error if passwords do not match.

In [ ]:
from pydantic import BaseModel, model_validator

class User(BaseModel):
    password: str
    confirm_password: str

    @model_validator(mode='after')
    def passwords_match(cls, model):
        if model.password != model.confirm_password:
            raise ValueError("Passwords do not match")
        return model

User(password="a", confirm_password="b")v

## Nested Models

Nested models allow Pydantic to handle hierarchical or complex data structures cleanly.

1. **class Address(BaseModel)**: Defines an address model with street and city.

2. **class UserProfile(BaseModel)**: Defines a user model with a nested Address field.

3. **address = Address(...)**: Creates an address instance.

4. **user = UserProfile(...)**: Creates a user instance, validating both user and nested address.

In [ ]:
from pydantic import BaseModel

class Address(BaseModel):
    street: str
    city: str

class UserProfile(BaseModel):
    name: str
    age: int
    email: str
    address: Address

address = Address(street="10 Rue de la Paix", city="Paris")
user = UserProfile(name="Emma Dubois", age=34, email="emma.dubois@example.fr", address=address)
print(user)

## Parsing & Serialization

Pydantic can parse JSON or other serialized data formats into Python objects automatically, validating types in the process.

- **data = '{"name": ...}'**: JSON string representing user data.

- **UserProfile.model_validate_json(data)**: Parses the JSON string and validates each field.

In [ ]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    name: str
    age: int
    email: str

data = '{"name": "Noah Kim", "age": 28, "email": "noah.kim@example.kr"}'
user = UserProfile.model_validate_json(data)
print(user)

You can convert a Pydantic model back to JSON for storage, APIs or communication with other systems.

- **UserProfile(...)**: Creates a Python object with validated data.

- **user.model_dump_json()**: Serializes the object into a JSON string.

In [ ]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    name: str
    age: int
    email: str

user = UserProfile(name="Luca Rossi", age=29, email="luca.rossi@example.it")
json_data = user.model_dump_json()
print(json_data)

Pydantic allows optional fields using Python’s Optional type which default to None if not provided. This is useful for fields that may be missing.

- **age: Optional[int] = None**: Declares age as optional, defaulting to None.

- **UserProfile(...)**: Creates an instance without the optional field.

In [ ]:
from typing import Optional
from pydantic import BaseModel

class UserProfile(BaseModel):
    name: str
    age: Optional[int] = None
    email: str

user = UserProfile(name="Mia Johansson", email="mia.johansson@example.se")
print(user)

## Settings Management

Pydantic provides BaseSettings to manage configuration via environment variables, .env files and other sources. This is useful for real-world apps where settings may change between environments.

- class Settings(BaseSettings): Defines a settings model that reads environment variables automatically.

- debug: bool = False: Optional field with a default value.

- class Config: env_file = ".env": Instructs Pydantic to also load values from a .env file.

- Settings(...): Instantiates the settings, validating all fields.

In [ ]:
from pydantic_settings import BaseSettings

class Settings(BaseSettings):
    app_name: str
    debug: bool = False
    database_url: str

    class Config:
        env_file = ".env"

settings = Settings(app_name="TestApp", database_url="sqlite:///:memory:")
print(settings.app_name)

## Custom Types

Pydantic supports constrained types and custom data classes to enforce stricter rules, reducing boilerplate validation.

- PositiveInt = conint(gt=0): Custom integer type greater than 0.

- name: constr(min_length=1): Ensures name is not an empty string.

- tags: List[str] = []: Optional list of tags with a default empty list.

- Product(...): Creates a validated product object.

In [ ]:
from pydantic import BaseModel, conint, constr
from typing import List

PositiveInt = conint(gt=0)  

class Product(BaseModel):
    name: constr( min_length=1 )  
    price: PositiveInt
    tags: List[str] = []

p = Product( name="Pen", price=10, tags=["stationery"] )
print(p)

## Aliasing & Field Customization


Pydantic allows you to set aliases, default values and metadata for fields. This is useful when your external data uses different field names or when you want to provide descriptive information for documentation.

- full_name: str = Field(..., alias="name"): Maps incoming name to internal full_name.

- age: int = Field(18, description="Age of the user"): Default value of 18 with metadata description.

- User(name="Rita"): Instantiates the model using the alias.

- user.model_dump(): Returns the model as a dictionary with internal field names.

In [ ]:
from pydantic import BaseModel, Field

class User(BaseModel):
    full_name: str = Field(..., alias="name")  # Alias for external data
    age: int = Field(18, description="Age of the user")  # Default and description

user = User(name="Rita")  # Uses alias to map 'name' to 'full_name'
print(user.model_dump())

## Strict Types


By default, Pydantic coerces types (e.g., "1" -> 1). Strict types prevent this coercion, ensuring that the input type matches exactly.

- id: StrictInt: Ensures the field only accepts integers.

- Account(id=5): Works fine.

- Account(id="5"): Raises a validation error because the input is a string.

In [ ]:
from pydantic import BaseModel, StrictInt

class Account(BaseModel):
    id: StrictInt

Account(id=5)         # valid
Account(id="5")       # invalid and raises ValidationError

## Error Handling and Custom Error Messages


Pydantic allows you to handle validation errors and customize messages using validators. This is particularly useful in APIs where you want to provide structured and clear error feedback.

- @field_validator('age'): Adds a custom check for the age field.

- raise ValueError(...): Raises an error if the value does not meet the criteria.

- try/except ValidationError: Catches validation errors for structured output.

- e.json(): Returns error details in JSON format for clients.

In [ ]:
from pydantic import BaseModel, ValidationError, field_validator

class User(BaseModel):
    name: str
    age: int

    @field_validator('age')
    def check_age(cls, v):
        if v < 18:
            raise ValueError('User must be 18 or older')
        return v

try:
    User(name="Sam", age=16)
except ValidationError as e:
    print(e.json())

## Performance and Optimizations


The key strategies to optimize Pydantic for speed and efficiency are as follows:

- Use model_validator to centralize checks when multiple fields interact.

- Use constrained types and strict types where appropriate to reduce validation code.

- For settings-heavy apps, instantiate settings once and reuse the object.

- Consider pydantic C extensions for micro-optimizations when parsing extremely large volumes.